# Tarea 4: Bases de datos 

Esta tarea se requerirá otra vez la paquetería <a href=" https://julianlsolvers.github.io/LsqFit.jl/latest/tutorial/">LsqFit</a>, revisa la documentación si no lo haz hecho.

También, puse un problema con propósito del 8-M, es un problema largo, pero por muy poco esfuerzo extra pueden tener además de los 0.7 que vale el problema, 2pts extra. Me importa que, aunque hacen en equipo la tarea, ese problema revísenlo todos en el equipo. Pido que discutan algo, discútanlo entre todos los del equipo e incluso, sería bueno que muestren los resultados a varias personas y pregunten su opinión. 
    
Para Instalar la paquetería, igual que con el resto hasta ahora, tienen que escribir en una celda:

] add LsqFit
    
Una vez instalada la cargan usando: 

using LsqFit

Además, para un par de los problemas avanzados requerirás hacer un Bot de Telegram, así que revisa los videos correspondientes también. 

Equipo:

- Andres Rashid Abdul Flores Niño
- Jose Angel Aké Jiménez
- Marco Antonio Herrera Solar

## Problemas Básicos

Antes de comenzar a trabajar en los ejercicios, necesitas conseguir datos de un experimento (de tus laboratorios o del laboratorio de algún amigo). Es necesario que el experimento sea suficientemente "complejo" en el siguiente sentido. Debe haber mediciones directas (como medir la intensidad de la luz, sonido, posición de algo, un ángulo de disperción, etc) y mediciones indirectas (es decir, cantidades que obtienes a partir de las mediciones directas, por ejemplo una velocidad, una longitud de onda, etc.) que obtengas mediante una fórmula. Casi todos los experimentos de los laboratorios de la facultad sirven para estos propósitos. Si estás haciendo una práctica, es muy probable que los datos que hayas recabado cumplan con estos criterios y puedas usar estos problemas del curso para matar dos pájaros de un tiro (disculpen la expresión), resolver la tarea y hacer unas bonitas gráficas con resultados buenos. 
    
[1] Utilizando DataFrames genera una base de datos **vacía** de algún experimento de labotartorio, poniendo sólo las columnas de lo que se mide directamente en el experimento y después llénala de datos. Por ejemplo, digamos que mediste el ángulo de refracción y reflexión de la luz como función del ángulo de insidencia. Entonces generarás un DataFrame que contenga como variables el ángulo de incidencia, ángulo de refracción y ángulo de reflexión. 

**Nota** Abre una celda de MarkDown para explicar brevemente en qué consiste el experimento. 
**Nota2** No necesita ser un experimento real, puedes poner ruido ficticio a los datos de una función de algo que deberías obtener experimentalmente. Sólo, en caso de que sean datos inventados, trata que sean realistas y sobre algún experimento interesante del que conozcas la teoría. 

In [ ]:
using LsqFit, CSV, DataFrames, Plots, Statistics, StatsBase, LinearAlgebra, ImageShow, StringEncodings, ColorSchemes, LaTeXStrings, Dates

In [ ]:
#Creamos un dataframe de cero
DataFrame()

He aquí el experimento para el cual se supone estaremos 'midiendo' los ángulos de incidencia, reflexión, transmisión:

[DETERMINACIÓN DE ÍNDICES DE REFRACCIÓN](https://fisicas.ucm.es/data/cont/media/www/pag-36931/Guiones16-17/prac22-1617.pdf)

A continuación mostramos el ejemplo de un DataFrame vacío.

In [ ]:
dtfv = DataFrame(θᵢ = [], θₜ = [], θᵣ = [])

*A continuación mostramos la tabla que usaremos para este y los siguientes ejercicio, pero el experimento se respeta y aparte se agregan otras medidas.*

In [ ]:
exprmt = DataFrame(CSV.File("Copia-de-P4_AcrilicoAire-copia.csv"))

In [ ]:
exprmt." Angulo refraccion" = coalesce.(exprmt." Angulo refraccion",NaN)

In [ ]:
exprmt

**3/5 pts (3/5 de 10)**

**Nota:** Bien! Les recomiendo quitar las celdas donde instalan paqueterías. 

[2] Con los datos del experimento se trata de obtener alguna otra medición, por ejemplo, el ángulo de refracción y reflexión usando directamente la ley de Snell y el dato del ángulo de incidencia. Agrega las columnas que faltan ya con los datos calculados. 

Algo importante a tomar en cuenta es el error directo en la medición, es decir, el erro del instrumento de medición. En el caso de lo que se mide directamente es un error típicamente constante, pero las mediciones indirectas su error de medición típicamente depende de la medición en sí, por lo que para cada valor hay que calcular el error, no olvides agregar la correspondiente columna o columans de los errores.  

**Nota:** Si tienes dudas de la propagación del error revisa https://es.wikipedia.org/wiki/Propagaci%C3%B3n_de_errores. Aunque es un tema que todos los físicos deberíamos dominar súper bien, no es verdad que lo dominemos y frecuentemente hacemos mal los cálculos (por bien que dominemos las derivadas). 

In [ ]:
insertcols!(exprmt,7,:"Sen Ang Inc" => sind.(exprmt."Angulo incidencia"))
insertcols!(exprmt,8,:"Inc Sen Ang Inc" => cosd.(exprmt."Angulo incidencia") .* exprmt."Inc Ang Inc") 
insertcols!(exprmt,9,:"Sen Ang Refraccion" => sind.(exprmt." Angulo refraccion"))
insertcols!(exprmt,10,:"Inc Sen Ang Refr" => cosd.(exprmt." Angulo refraccion") .* exprmt."Inc Ang Refraccion") 

**3/5 pts (6/5 de 10)**

**Nota:** Bien!, si les interesa automatizar la propagación de errores con Julia, les recomiendo revisar la paquetería [Measurements.jl](https://github.com/JuliaPhysics/Measurements.jl).

[3] Haz una descripción general de tus datos (promedio, valor mínimo, valor máximo y desviación estandar).

In [ ]:
describe(exprmt,:mean,:min,:max,:std)

**3/5 pts (9/5 de 10)**

[4] Usa la paquetería LsqFit para ajustar la(s) curva(s) teórica(s) que deberías observar en los datos que obtuviste (por ejemplo, ajusta con la ley de Snell dejando libre el parámetro del índice o índices de refracción, para que con el ajuste obtengas el índice de refracción). Interpreta los parámetros que obtengas del ajuste (por ejemplo, el que obtienes el índice de refracción). Obtén también los errores en el ajuste, por ejemplo, cual es el error en tu medición del índice de refracción. 

In [ ]:
a = filter(x -> !isnan(x),exprmt[!,5])

In [ ]:
df = DataFrame(θᵢ = exprmt."Angulo incidencia"[1:10] , θᵣ = a)

In [ ]:
m(t,p) = p[1] .* t
P₀ = [2/3]
fit = curve_fit(m, sind.(df.θᵢ), sind.(df.θᵣ), P₀)

In [ ]:
fit.param

**3/5 pts (12/5 de 10)**

**Notas:**
- Les recomiendo desglosar lo que hicieron, en este caso tuvieron que ajustar `sin` vs `sin` ¿por qué?
- El parámetro que obtienen ¿qué representa? ¿tiene sentido físico?
- ¿Cuál sería el error de su ajuste?

[5] Haz una gráfica de tus datos, variables independientes vs variables dependientes. Si hay varias variables independientes, fija todas excepto 2 (para hacer una gráfica 3D por cada variable dependiente). Si tienes muchas variables, elije la (o las) combinación(es) que consideres importante(s) y en epecial, la(s) que incluya(n) el(los) ajuste(s) teórico(s) que hayas obtenido en el problema anterior e incluye en esa(s) gráfica(s) el plot de tu curva teórica. Pon "bonitos" los ejes, etiquetas, etc. (es decir, utiliza LaTeXStrings para poner los labels que requieras).

In [ ]:
scatter(sind.(df[!,1]),sind.(df[!,2]),label = "")
plot!(x -> m(x,fit.param),0,0.7, label = "", xlabel = L"sin($\theta_i$)", ylabel = L"sin($\theta_r$)")

**3/5 pts (3 de 10)**

**Nota:** Bien! ¿Cómo graficarían sus puntos con barras de error?

[6-7] (este ejercicio vale 2 porque implica algo más de trabajo) Descarga de <a href=" https://datos.covid-19.conacyt.mx/#DownZCSV">aquí</a> la base de datos de muertes por Covid por municipio y la de casos de Covid por municipio. Carga las respectivas bases de datos y con ellas genera 2 imágenes y dos diccionarios para leer las imágenes. Las imágenes deben contener una base de datos donde cada pixel represente la taza de mortalidad o de casos de covid de cada municipio. Es decir, el pixel $[i,j]$ representa la tasa de mortalidad (la cantidad de muertes dividido entre la población de la entidad) del municipio $i$ el día $j$ de la pandemia. Los diccionarios deben servir para poder descomprimir adecuadamente estas imágenes. Además de estos dos diccionarios, conviene generar otros 2 diccionarios más, para encontrar qué municipio es el municipio $i$ y para encontrar a qué fecha se refiere el día $j$. El de los municipios es igual para ambas bases de datos, pero el de las fechas no. 

¿Puedes ver en las imágenes cuándo fueron cada una de las olas? ¿Notas municipios (regiones) que sigan un comportamiento diferente al de la mayoría de los municipios en el país? 

**Nota:** Hay municipios con población igual a $0$. Esos municipios se debe de poner una tasa de mortalidad o de casos igual a $0$. 

**Nota2:** Para visualizar mejor tus datos, asegúrate que el cambio de colores sea suave. No que de "0.1" tengas un rojo y "0.1001" tengas un verde. Esto no es obligatorio, pero es mejor si lo haces bien. 

In [ ]:
# Cargamos las bases de datos de casos por municipio
Casos_diarios_municipio = DataFrame(CSV.File("Casos_Diarios_Municipio_Confirmados_20230313.csv")) 

In [ ]:
#Bases de datos de defunciones por municipio
Muertes_diarias_municipio = DataFrame(CSV.File("Casos_Diarios_Municipio_Defunciones_20230307.csv"))

In [ ]:
#Funcion para calcular la tasa de contagios/muertes por covid dado un dataframe, dia y municipio
function tasa_covid(dataframe,i,j)
    if dataframe[i,2] == 0
        return 0
    else
        return dataframe[i,j]/dataframe[i,2]
    end
end

In [ ]:
#Verifica si la poblacion de algun municipio es distinta de cero
function poblacion_distinta_de_cero(dataframe,i)
    if dataframe[i,2] == 0
        return 0
    else
        return 1
    end
end

In [ ]:
#Funcion para convertir cada parte del dataframe en una imagen bajo una regla de correspondencia
function codificar_pixel(pixel,i,j,dataframe) 
    if poblacion_distinta_de_cero(dataframe,i) == 0 # Si la población del grupo es cero, entonces el pixel es 0
        pixel[i,j] = RGB{N0f8}(0,0,0)
    else 
        pixel[i,j] = RGB(float(tasa_covid(dataframe,i,j))*10^6,0.1-float(tasa_covid(dataframe,i,j))*10^6,0.1-float(tasa_covid(dataframe,i,j))*10^6)
    end
    
    return pixel[i,j]
end

In [ ]:
#=
Creamos una función (general) que dado una imagen, un dataframe, un diccionario y una regla de correspondencia
devuelve una imagen
=#
function dataframe_a_imagen(funcion_generadora, dataframe)
        fondo = fill(RGB(0.0,0.0,0.0), nrow(dataframe), ncol(dataframe))
        fondo = [funcion_generadora(fondo,i,j,dataframe) for i in 1:nrow(dataframe), j in 4:ncol(dataframe)]
    return fondo
end

In [ ]:
imagen_casos_diario_municipio = dataframe_a_imagen(codificar_pixel,Casos_diarios_municipio)

In [ ]:
imagen_muertes_diarias_municipio = dataframe_a_imagen(codificar_pixel,Muertes_diarias_municipio)

En ambas imágenes (especialmente la segunda) se pueden apreciar apreciar cada una de las olas de COVID dadas por distintas
franjas de color rojo, las cuales tienen franjas intermedias entre sí. A mayor cantidad de rojo en cada pixel se indica mayor
cantidad de muertes. Por otro lado, si vemos la imagen de arriba a abajo, veremos que existen franjas horizontales rojizas, 
algunas con mayor intensidad que otras, las cuales representan los municipios/estados con mayor cantidad de muertes por COVID 
(esto se nota especialmente en la imagen para los casos y no tanto para las muertes).

In [ ]:
#Creamos los diccionarios para descomprimir las imágenes

#Diccionario_para_fechas
diccionario_fechas_muertes = Dict{Int64,String}(i-3 => names(Muertes_diarias_municipio)[i] for i in 4:size(Muertes_diarias_municipio,2) )
diccionario_fechas_casos = Dict{Int64,String}(i-3 => names(Casos_diarios_municipio)[i] for i in 4:size(Casos_diarios_municipio,2) )
#Diccionarios para poblaciones de los municipios, nombres y claves asociadas
diccionario_poblacion_municipios = Dict{Int64,Int64}(i => Casos_diarios_municipio[i,2] for i in 1:size(Casos_diarios_municipio,1) )
diccionario_claves_municipios = Dict{Int64,Int64}(i => Casos_diarios_municipio[i,1] for i in 1:size(Casos_diarios_municipio,1) )
diccionario_nombres_municipios = Dict{Int64,String}(i => Casos_diarios_municipio[i,3] for i in 1:size(Casos_diarios_municipio,1) );

In [ ]:
#Obtenemos a partir de un pixel de la imagen la tasa de contagios/muertes
function datos_por_fecha_poblado(imagen,i,j,poblacion)
    return ceil((imagen[i,j].r)*poblacion[i]/10^6 )
end

In [ ]:
#=
function transforma_imagen_en_tabla(imagen,diccionarios)
    #Creamos las columnas que corresponden a la poblacion y a los nombres de los municipios
    columna = []
    for i in 1:length(diccionarios)
        columna[i] = [diccionarios[i][j] for j in 1_:length(diccionarios[i])]
    end
    #=
    columna_poblacion = [diccionario_poblacion_municipios[i] for i in 1:length(diccionario_poblacion_municipios) ]
    columna_nombres_municipios = [diccionario_nombres_municipios[i] for i in 1:length(diccionario_indices_municipios)]
    columna_claves_municipios = [diccionario_claves_municipios[i] for i in 1:length(diccionario_poblacion_municipios)]
    =#
    tabla = DataFrame([columna_claves_municipios,columna_poblacion, columna_nombres_municipios],["Clave","Poblacion", "Municipio"])
    for i in 1:length(diccionario_fechas)
        columna_dia = [datos_por_fecha_poblado(imagen,j,i,columna_poblacion) for j in 1:length(diccionario_indices_municipios)]
        tabla[!,diccionario_fechas[i]] = columna_dia
    end
    
    return tabla
end
=#

In [ ]:
#Con la funcion anterior transformamos la imagen en tabla
function transforma_imagen_en_tabla(imagen,diccionario_poblacion_municipios,diccionario_fechas,diccionario_indices_municipios, diccionario_claves_municipios)
    #Creamos las columnas que corresponden a la poblacion y a los nombres de los municipios
    columna_poblacion = [diccionario_poblacion_municipios[i] for i in 1:length(diccionario_poblacion_municipios) ]
    columna_nombres_municipios = [diccionario_nombres_municipios[i] for i in 1:length(diccionario_indices_municipios)]
    columna_claves_municipios = [diccionario_claves_municipios[i] for i in 1:length(diccionario_poblacion_municipios)]
    tabla = DataFrame([columna_claves_municipios,columna_poblacion, columna_nombres_municipios],["Clave","Poblacion", "Municipio"])
    for i in 1:length(diccionario_fechas)
        columna_dia = [datos_por_fecha_poblado(imagen,j,i,columna_poblacion) for j in 1:length(diccionario_indices_municipios)]
        tabla[!,diccionario_fechas[i]] = columna_dia
    end
    
    return tabla
end

In [ ]:
#Generamos una nueva tabla para los casos diarios
nueva_tabla_casos_diarios = transforma_imagen_en_tabla(imagen_casos_diario_municipio,diccionario_poblacion_municipios,diccionario_fechas_casos,diccionario_nombres_municipios, diccionario_claves_municipios);
nueva_tabla_casos_diarios[1:5, 1:5]

In [ ]:
Casos_diarios_municipio[1:5, 1:5]

In [ ]:
#De manera similar con el caso de las muertes 
nueva_tabla_muertes_diarias = transforma_imagen_en_tabla(imagen_muertes_diarias_municipio, diccionario_poblacion_municipios,diccionario_fechas_muertes,diccionario_nombres_municipios,diccionario_claves_municipios);
nueva_tabla_muertes_diarias[1:5, 1:5]

In [ ]:
Muertes_diarias_municipio[1:5, 1:5]

**6/5 pts (21/5 de 10)**

**Nota:** Excelente! Un detalle, Asignar memoria con
```julia
fondo = fill(RGB(0.0,0.0,0.0), nrow(dataframe), ncol(dataframe))
```
no tiene mucho sentido si luego vuelven a crear el objeto con 
```julia
fondo = [funcion_generadora(fondo,i,j,dataframe) for i in 1:nrow(dataframe), j in 4:ncol(dataframe)]
```
Lo ideal sería asignar memoria (con la primera línea de código) y luego modificar `fondo` con un `for`.

[8] Utiliza tus imágenes y funciones anteriores para ahora graficar las tasas de casos de covid en los 3 municipios donde la tasa de casos haya sido la más alta en algún día. Repite lo mismo para las tasas de mortalidad por covid. 

In [ ]:
function hallar_municipios_con_mayor_tasa(imagen, dia, diccionario_poblacion,diccionario_fechas,diccionario_nombres,diccionario_claves)
    tabla = transforma_imagen_en_tabla(imagen, diccionario_poblacion,diccionario_fechas,diccionario_nombres,diccionario_claves)
    columna_poblacion = [diccionario_poblacion_municipios[i] for i in 1:length(diccionario_poblacion) ]
    #datos_por_fecha_poblado(imagen,i,j,columna_poblacion)
    tasa_municipios = sort([tabla[i,dia+3] for i in 1:size(tabla,1)], rev=true)
    indices_municipios_con_mas_casos = indexin(tasa_municipios[1:3],tabla[:,dia+3])
    #municipios_con_mas_casos = [diccionario_nombres[i] for i in indices_municipios_con_mas_casos]
    tasa_de_casos = [tabla[i,dia+3] for i in indices_municipios_con_mas_casos ]
    #return municipios_con_mas_casos, tasa_de_casos
    return indices_municipios_con_mas_casos
end

In [ ]:
#Hallamos los municipios con mayor tasa de muertes dado un día
municipios_con_mayor_tasa_de_muertes = hallar_municipios_con_mayor_tasa(imagen_muertes_diarias_municipio,130 ,diccionario_poblacion_municipios,diccionario_fechas_muertes,diccionario_nombres_municipios,diccionario_claves_municipios)

In [ ]:
#Funcion que grafica los municipios con mayor tasa en un dia dado a traves del tiempo
function grafica_municipios_con_mayor_tasa_dia(imagen, dia, diccionario_poblacion,diccionario_fechas,diccionario_nombres,diccionario_claves)
    indices_municipios = hallar_municipios_con_mayor_tasa(imagen, dia, diccionario_poblacion,diccionario_fechas,diccionario_nombres,diccionario_claves)
    tabla = transforma_imagen_en_tabla(imagen, diccionario_poblacion,diccionario_fechas,diccionario_nombres,diccionario_claves)
    dias = length(diccionario_fechas)
    x = range(1,dias,length = length(diccionario_fechas))
    tasas = [[tabla[i,j] for j in 4:dias] for i in indices_municipios]
    graficas = []
    for i in 1:length(tasas)
        push!(graficas, scatter(x, tasas[i], title= diccionario_nombres[indices_municipios[i]], legend=false))
    end
    plot(graficas...)
    # scatter(x, [tasas[i] for i in 1:length(tasas)], title = diccionario_nombres[i] for i in indices_municipios, layout=(3,1), legend=false)
end

In [ ]:
#En este caso graficamos para los municipios con mayor tasa de contagios el dia 100 de la pandemia
grafica_municipios_con_mayor_tasa_dia(imagen_casos_diario_municipio,100,diccionario_poblacion_municipios,diccionario_fechas_casos,diccionario_nombres_municipios,diccionario_claves_municipios)

In [ ]:
grafica_municipios_con_mayor_tasa_dia(imagen_muertes_diarias_municipio,100,diccionario_poblacion_municipios,diccionario_fechas_muertes,diccionario_nombres_municipios,diccionario_claves_municipios)

**3/5 pts (24/5 de 10)**

**Notas:** Muy bien!
- Les recomiendo reducir el tamaño de los puntos en `scatter` para que se aprecie mejor la curva. 
- En su función `hallar_municipios_con_mayor_tasa` definen `tasa_de_casos` pero nunca la usan. 

[9] Haz una convolución de tu imagen, donde el kernel sea: $$\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
1/7 & 1/7 & 1/7 & 1/7 & 1/7 & 1/7 & 1/7\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$

después con la nueva imagen obtén los casos de covid para los mismos 3 municipios y grafica el número de casos de covid como función del tiempo. 

¿Qué es lo que hicieste con esa convolución?

**Nota:** otra opción de convolución buena es: $$\frac{1}{98}\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 6 & 24 & 36 & 24 & 6 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0
\end{pmatrix}$$

In [ ]:
#=
Primero definimos unas funciones de manera global que 
utilizaremos en este ejercicio y el siguiente
=#

function obtener_color(pixel,i,j,k,l,color) #Obtener el color de un pixel en específico
    if color == "rojo"
        return pixel[i+k,j+l].r #Obtenemos rojo
    elseif color == "azul"
        return pixel[i+k,j+l].b #Obtenemos azul
    elseif color == "verde"
        return pixel[i+k,j+l].g #Obtenemos verde
    else
        error("No permitido")
    end
end
    
function evaluar_pixel(pixel,i,j,k,l,color) #Vemos si el pixel se encuentra definido (i.e. si no está fuera de los bordes)
    if i+k<=0 || j+l<=0 || i+k> size(pixel,1) || j+l> size(pixel,2)
        return 0 #Si no es así, su valor es cero
    else
        return obtener_color(pixel,i,j,k,l,color) #En otro caso obtenemos el color
    end
end

#funcion para la convolucion de un pixel dado un kernel
function kernel_convolucion(pixel,kernel,x,y,n,color)
    #Obtiene la convolucion de una vecindad de (2n+1,2n+1) por color
    if color == "rojo"
        suma = sum([evaluar_pixel(pixel,x,y,i,j,"rojo")*kernel[n+1+i,n+1+j] for i in -n:n, j in -n:n])
    elseif color == "verde"
        suma = sum([evaluar_pixel(pixel,x,y,i,j,"verde")*kernel[n+1+i,n+1+j] for i in -n:n, j in -n:n])
    elseif color == "azul"
        suma = sum([evaluar_pixel(pixel,x,y,i,j,"azul")*kernel[n+1+i,n+1+j] for i in -n:n, j in -n:n])
    else
        error("No permitido")
    end
    
    #Evaluamos por casos la suma de la convolucion obtenida
    if suma < 0
        return 0
    elseif suma > 1
        return 1
    else 
        return suma 
    end 
end

function convolucion(imagen,kernel,n) #Funcion principal de convolucion, 'n' es la n del kernel (2n+1, 2n+1)
    
    #Obtenemos los colores convolucionados de cada pixel de la imagen
    rojo_convolucionado = [kernel_convolucion(imagen,kernel,i,j,n,"rojo") for i in 1:size(imagen,1), j in 1:size(imagen,2)]
    azul_convolucionado = [kernel_convolucion(imagen,kernel,i,j,n,"verde") for i in 1:size(imagen,1), j in 1:size(imagen,2)]
    verde_convolucionado = [kernel_convolucion(imagen,kernel,i,j,n,"azul") for i in 1:size(imagen,1), j in 1:size(imagen,2)]

    #Juntamos todos los colores convolucionados en una sola imagen para obtener la imagen convolucionada
    imagen_convolucionada = [RGB(rojo_convolucionado[i,j], verde_convolucionado[i,j],azul_convolucionado[i,j]) for i in 1:size(imagen,1), j in 1:size(imagen,2)]
    return imagen_convolucionada
end

In [ ]:
#Obtenemos la convolucion del primer kernel
kernel = zeros(7,7)
kernel[4,:] = [1/7 for i in 1:7] 
imagen_convolucionada_kernel_1_casos_diarios = convolucion(dataframe_a_imagen(codificar_pixel,Casos_diarios_municipio),kernel,3)

In [ ]:
#Graficamos los municipios con mayor tasa de contagios obtenida a partir de la imagen convolucionada
grafica_municipios_con_mayor_tasa_dia(imagen_convolucionada_kernel_1_casos_diarios,100,diccionario_poblacion_municipios,diccionario_fechas_casos,diccionario_nombres_municipios,diccionario_claves_municipios)

In [ ]:
#Obtenemos la convolucion del segundo kernel
kernel_2 = zeros(7,7)
kernel_2[4,:] = (1/98)*[1,6,24,36,24,6,1]
imagen_convolucionada_kernel_2_casos_diarios = convolucion(dataframe_a_imagen(codificar_pixel,Casos_diarios_municipio),kernel_2,3)

In [ ]:
grafica_municipios_con_mayor_tasa_dia(imagen_convolucionada_kernel_2_casos_diarios,100,diccionario_poblacion_municipios,diccionario_fechas_casos,diccionario_nombres_municipios,diccionario_claves_municipios)

Intuitivamente lo que estamos haciendo con estos Kernels es promediar las tasas de casos/muertes respecto
a los días y municipios colindantes. Esto como resultado da que al momento de graficar las tasas de casos/muertes se obtengan valores que asemejan a rectas horizontales.

**3/5 pts (27/5 de 10)**

**Nota:** Su función para encontrar los municipios con más casos devuelve tres veces Zapopan, lo cual no debería pasar. 

[10] **Problema especial por el Día Internacional De La Mujer**. El día de la mujer conmemora una serie de eventos donde se reprimieron movimientos feministas que exigían los mismos derechos que los hombres y de paso recuerda las diferencias que aún persisten en la sociedad debido al género. En este problema investigarán un poco sobre estas diferencias y tratarán de correlacionarlo con algunas posibles causales. 

Utiliza las bases de datos de (que vas a tener que limpiar): 

https://www.gob.mx/sesnsp/acciones-y-programas/datos-abiertos-de-incidencia-delictiva?state=published

https://ilostat.ilo.org/data/

https://population.un.org/wpp/Download/Standard/MostUsed/

https://genderdata.worldbank.org/indicators/se-sch-life/?view=correlation


Para estudiar la relación entre (es decir, haz las gráficas x vs y y trata de ajustar alguna función):

(i) "Violencia familiar y violaciones" en México para varios meses y estados (cada mes-estado es un punto de la gráfica que después tratarás de ajustar con alguna función, por ejemplo una recta, una ley de potencias, etc... esto si es que visualmente se observa que hay alguna correlación entre los datos). ¿Ha mejorado la situación en los últimos años? (para responder esto último grafica violaciones vs tiempo para el país completo), ¿Dónde están son los principales violadores?¿Qué se puede hacer para reducir eso? (ver nota2). 

(ii) Gap en el salario vs producto interno bruto per cápita de cada país. En este caso normaliza el pib percápita con el pib percápita del país más rico. Una vez normalizado. Repite el ejercicio para todos los años del 2009 al 2016. En cada año hay que normalizar con los datos de ese año y ajusta una recta. Finalmente haz un promedio de las pendientes.  ¿Qué países discriminan más a las mujeres económicamente, los ricos, los pobres o es parejo? ¿Qué puede ser la causa de esto? (ver el siguiente inciso)  

(iii) Porcentaje de mujeres en trabajos de paga baja y el producto interno bruto del país. 

Una de las cosas que causa este problema económico es que en los puestos altos hay muchas menos mujeres (uno de las diferencias que pueden analizar para los puntos extra). Entre más rico es el país, más aportan los puestos altos a la economía. ¿Qué podríamos hacer para reducir esto? (ver nota 2).

(iv) Mortalidad infantil vs años promedio de educación en mujeres. ¿Qué tan importante es la educación de las mujeres para la sobrevivencia de los niños en los primeros años de vida? Compara con producto interno bruto per cápita. 

Esto probablemente significa que realmente recae mucho la responsabilidad del cuidado de los bebés en las mujeres. ¿Cómo se podría cambiar eso? (si las mujeres se tienen que ocupar de los hijos, significa que no pueden crecer profesionalmente, lo cual también significa que no tienen acceso a los puestos altos, etc...) 

**Nota: por 2pto extra (en la tarea), investiga alguna de las diferencias que no se revisaron en este problema y revisa si hay correlación con por lo menos 2 posibles variables que consideres son parte de la causal (puedes usar otras bases de datos, sólo pon el link de donde las bajaste)** 

**Nota2: Terminado este problema, discute (un poco) sobre lo que puedes hacer para reducir las diferencias por género ¿qué puedes hacer para tener un impacto de gran escala? (uno puede tener efectos de corto alcance, como repartir las labores domésticas más justamente, pero también se puede hacer algunas cosas para tener un efecto a gran escala, como divulgación en escuelas, blogs, presión política, etc... donde el impacto individual quizá es menor, pero el impacto total es mucho mayor).**

(i) "Violencia familiar y violaciones" en México para varios meses y estados

In [ ]:
Crimen = CSV.File(open(read, "IDEFC_NM_ene23.csv", enc"ISO-8859-1")) |> DataFrame;

In [ ]:
meses = ["Ene", "Feb", "Mar", "Abr", "May", "Jun", "Jul", "Ag", "Sep", "Oct", "Nov", "Dic"];

In [ ]:
tipo_de_delito = unique(Crimen."Tipo de delito");

In [ ]:
entidad = unique(Crimen."Entidad")
año = unique(Crimen."Año")
ndelitos = length(tipo_de_delito) 
naños = length(año)
nentidades = length(entidad)
casos = zeros(12*naños, ndelitos, nentidades);

In [ ]:
IndexDelito = [[] for i in 1:ndelitos]
IndexEntidad = [[] for i in 1:nentidades]
IndexA = [[] for i in 1:naños];

In [ ]:
for i in 1:ndelitos
    if mod(i,4) == 0
        println(i/40*100, "%")
    end
    IndexDelito[i] = findall(x-> x == tipo_de_delito[i], Crimen."Tipo de delito")
    for j in 1:nentidades
        IndexEntidad[j] = findall(x->x == entidad[j], Crimen."Entidad"[IndexDelito[i]])
        for k in 1:naños
            IndexA[k] = findall(x-> x == año[k], Crimen.Año[IndexDelito[i]][IndexEntidad[j]])
        end
    end
end

In [ ]:
findfirst(x -> x == "Violencia familiar", tipo_de_delito)

In [ ]:
IndexDelito[24][224 + 1] #A partir de 2022 (32*7=224 corresponde a todos los meses antes de 2022)

In [ ]:
IndexDelito[24][224 + 2] - IndexDelito[24][224 + 1] #Cada cuanto aparece el valor que nos interesa 

In [ ]:
for i in 0:8
    indx = IndexDelito[24][32*i + 1]
    p = plot(title="Violencia Familiar", xlabel="Mes", ylabel="Casos")
    for k in 0:31
        a = collect(values(Crimen[indx + 98*k, 8:end]))
        scatter!(meses, a, label=false, markersize=3, colorbar=true)
    end
    display(p)
end

In [ ]:
findfirst(x -> x == "Violación simple", tipo_de_delito)

In [ ]:
for i in 0:8
    indx = IndexDelito[13][32*i + 1]
    p = plot(title="Violación simple", xlabel="Meses", ylabel="Número de violaciones")
    for k in 0:31
        a = collect(values(Crimen[indx + 98*k, 8:end]))
        scatter!(meses, a, label=false, markersize=3, colorbar=true)
    end
    display(p)
end

In [ ]:
lista = []
for i in 0:8
    indx = IndexDelito[13][32*i + 1]
    for k in 0:31
        push!(lista, collect(values(Crimen[indx + 98*k, 8:end])))
    end
end

In [ ]:
sumas_por_mes = [[],[],[],[],[],[],[],[]]
saltos = [i for i in 1:32:256]
for i in 1:7
    sumas_por_mes[i] = zeros(12)
    for mes in 1:12
        valores_del_mes = []
        for j in saltos[i]:saltos[i+1]-1
            push!(valores_del_mes, lista[j][mes])
        end
        sumas_por_mes[i][mes] = sum(valores_del_mes)
    end
end

In [ ]:
valores_totales_15_22 = []
for i in 1:8
    valores_totales_15_22 = vcat(valores_totales_15_22, sumas_por_mes[i]) 
end

In [ ]:
valores_totales_15_22 # = vcat(sumas_por_mes[1], sumas_por_mes[2], sumas_por_mes[3], sumas_por_mes[4], sumas_por_mes[5], sumas_por_mes[6], sumas_por_mes[7], sumas_por_mes[8]);

In [ ]:
lista_meses = [] #Se hace un for para etiquetar el eje x acorde al mes y año. i.e 0816 = Agosto 2016

for año in 15:22
    for mes in 1:12
        push!(lista_meses, "$(mes < 10 ? "0$mes" : mes)$(año < 10 ? "0$año" : año)")
    end
end

println("Lista de meses: ", lista_meses)

In [ ]:
palettecasos = distinguishable_colors(96); #Generamos un color para cada mes de cada año

In [ ]:
scatter(lista_meses, valores_totales_15_22, color = palettecasos, size=(1200, 400), legend = false, xlabel="Años esperados de estudio, mujeres", ylabel="Mortalidad infantil", title="Años de estudio y mortalidad infantil en varios países, 2010", titlefont=font(8))

Se pueden hacer demasiadas cosas para intentar reducir los casos de violación, los que más discutimos fueron:

Fortalecer el marco legal: es necesario revisar y actualizar las leyes existentes para garantizar que las penas por delitos sexuales sean más severas.

Mejorar el acceso a servicios de apoyo: se deben proporcionar servicios de apoyo gratuitos y confidenciales para las víctimas de violación, incluyendo asesoramiento psicológico y asistencia legal. ESto para todas las personas, independientemente de su género, edad, orientación sexual, origen étnico o situación económica.

Fomentar la denuncia: es necesario eliminar el estigma y la vergüenza asociados con la violación para que las víctimas se sientan seguras al denunciar y recibir apoyo.

ii) Gap en el salario vs producto interno bruto per cápita de cada país.

In [ ]:
GAP = CSV.File(open(read, "DP_LIVE_13032023221129884.csv")) |> DataFrame;

In [ ]:
GDP_P = CSV.File(open(read, "API_NY.GDP.PCAP.CD_DS2_en_csv_v2_4901665.csv")) |> DataFrame;

In [ ]:
#Aquí buscamos que valore que países hay en una lista y en la otra.
unique_gap = unique(GAP[:, "LOCATION"]); 

unique_gdp = unique(GDP_P[:, "Country Code"]);

#Ya sabiendo qué países hay en cada lista, podemos filtrar tal que nos quedemos con los países que comparten ambas listas

gap_filtrado = filter(row -> row."LOCATION" in unique_gdp, GAP);
gdp_filtrado = filter(row -> row."Country Code" in unique_gap, GDP_P);

In [ ]:
gdp_solo_columnas_interes = select(gdp_filtrado, [:"Country Code", :"2009", :"2010", :"2011", :"2012", :"2013", :"2014", :"2015", :"2016"])
gap_solo_columnas_interes = select(gap_filtrado, [:"LOCATION", :"TIME", :"Value"]);

In [ ]:
function gap_vs_pib(año)
    gap_a = filter(row -> row."TIME" == año, gap_solo_columnas_interes) #Seleccionamos unicamente los valores del año en cuestión
    valores_gap_a = collect(gap_a[:, "Value"]) #Los colocamos en una lista
    pais10 = [] #Aquí creamos una lista de los países que hay para tal año
    for row in eachrow(gap_a)
        push!(pais10, row."LOCATION")
    end
    gdp_filtrado_10 = filter(row -> row."Country Code" in pais10, gdp_solo_columnas_interes) #Entonces ya podemos obtener los valores para esos países
    valores_gdp_2010 = collect(gdp_filtrado_10[:, string(año)]) #Los recolectamos
    valores_gdp_2010_normalizados = valores_gdp_2010 ./ maximum(valores_gdp_2010) #Normalizamos los valores respecto al mas grande.
    scatter(valores_gdp_2010_normalizados, valores_gap_a)
    #Hacemos el ajuste de una recta y graficamos
    model(valores_gdp_2010_normalizados, p) = p[1]*valores_gdp_2010_normalizados .+ p[2]
    p0 = [1.0, 0.0]
    fit = curve_fit(model, valores_gdp_2010_normalizados, valores_gap_a, p0)
    p = fit.param
    plot!(valores_gdp_2010_normalizados, p[1]*valores_gdp_2010_normalizados .+ p[2], label="Recta ajustada")
    return plot!()
end

In [ ]:
gap_vs_pib(2009) #Podemos graficar para culquier año entre 2009-2016

In [ ]:
#Hacemos lo mismo pero que ahora nos regrese la pendiente del ajuste
function gap_vs_pib_pen(año)
    gap_a = filter(row -> row."TIME" == año, gap_solo_columnas_interes)
    valores_gap_a = collect(gap_a[:, "Value"])
    pais10 = []
    for row in eachrow(gap_a)
        push!(pais10, row."LOCATION")
    end
    gdp_filtrado_10 = filter(row -> row."Country Code" in pais10, gdp_solo_columnas_interes)
    valores_gdp_2010 = collect(gdp_filtrado_10[:, string(año)])
    valores_gdp_2010_normalizados = valores_gdp_2010 ./ maximum(valores_gdp_2010)
    scatter(valores_gdp_2010_normalizados, valores_gap_a)
    model(valores_gdp_2010_normalizados, p) = p[1]*valores_gdp_2010_normalizados .+ p[2]
    p0 = [1.0, 0.0]
    fit = curve_fit(model, valores_gdp_2010_normalizados, valores_gap_a, p0)
    p = fit.param
    return p[1]
end

In [ ]:
#Juntamos todos los valores de las pendientes para cada año solicitado
pendientes = []
for year in 2009:2016 
    push!(pendientes, gap_vs_pib_pen(year))
end

In [ ]:
mean(pendientes)

La pendiente positiva que que se obtuvo significa que a medida que aumenta el PIB per cápita de un país, el gap en el salario
entre hombres y mujeres disminuye. Esto nos podría dar la idea de que en países más ricos, las mujeres tienen menores oportunidades de acceso al mercado laboral y no pueden tener un salario más cercano al de los hombres.

Una posible explicación de lo que obtuvimos es que en países con economías más desarrolladas y mayor PIB per cápita, es más probable que haya sectores económicos dominados por hombres que generen ingresos más altos. Por ejemplo, en algunos países, los trabajos en el sector tecnológico, de ingeniería y finanzas suelen estar dominados por hombres, mientras que los trabajos en el sector de la educación o la atención sanitaria suelen estar dominados por mujeres. Estos trabajos a menudo están peor remunerados, lo que contribuye a la brecha salarial.

Otra posible explicación es que en países con altos niveles de desigualdad económica, es más probable que haya una mayor brecha salarial entre hombres y mujeres. Esto se debe a que las personas más ricas y poderosas a menudo tienen más influencia en el PIB.

###### eatherich

(iii) Porcentaje de mujeres en trabajos de paga baja y el producto interno bruto del país. 

In [ ]:
GDP = CSV.File(open(read, "API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4901850.csv")) |> DataFrame;

In [ ]:
MPOV = CSV.File(open(read, "EAR_XFLS_NOC_RT_A.csv")) |> DataFrame;

In [ ]:
unique_gdp = unique(GDP[:, "Country Code"]);

unique_mpov = unique(MPOV[:, "ref_area"]);

gdp_filtrado = filter(row -> row."Country Code" in unique_mpov, GDP);

In [ ]:
mpov_filtrado = filter(row -> row."ref_area" in unique_gdp, MPOV);

In [ ]:
moda = mode(mpov_filtrado[:, :"time"])

In [ ]:
mpov_2018 = filter(row -> row."time" == 2018, mpov_filtrado);

In [ ]:
unique_mpov_2018 = unique(mpov_2018[:, "ref_area"]);

In [ ]:
gdp_filtrado = filter(row -> row."Country Code" in unique_mpov_2018, GDP);

In [ ]:
valoresgdp = collect(gdp_filtrado[:,"2018"]);

In [ ]:
valoresmpov = collect(mpov_2018[:, "obs_value"]);

In [ ]:
scatter(valoresgdp, valoresmpov, color=distinguishable_colors(93), legend=false, xlabel="PIB por país", ylabel="Porcentaje mujeres, trabajos paga baja", title="Porcentaje de mujeres en trabajos de paga baja y el producto interno bruto del país, 2018", titlefont=font(7))

Algo que se podría hacer son políticas que permitan tanto a las madres como a los padres tomarse tiempo libre remunerado para cuidar de los hijos. Esto puede ayudar a reducir la discriminación contra las mujeres en el mercado laboral.

También se pueden ofrecer incentivos fiscales a las empresas que promuevan la igualdad de género y la inclusión laboral de las mujeres.

(iv) Mortalidad infantil vs años promedio de educación en mujeres.

In [ ]:
EXP = CSV.File(open(read, "Expected years of schooling.csv")) |> DataFrame;

EXP = select(EXP, [:"Economy", :"School life expectancy, primary to tertiary, female (years)"]);

EXP = dropmissing(EXP, :"School life expectancy, primary to tertiary, female (years)");

In [ ]:
MOR = CSV.File(open(read, "API_SH.DYN.MORT_DS2_en_csv_v2_4904013.csv")) |> DataFrame;

MOR = select(MOR, [:"Country Name", :"2010"]);

MOR = dropmissing(MOR, :"2010");

In [ ]:
unique_exp = unique(EXP[:, "Economy"]);

unique_mor = unique(MOR[:, "Country Name"]);

In [ ]:
exp_filtrado = filter(row -> row."Economy" in unique_mor, EXP);

mor_filtrado = filter(row -> row."Country Name" in unique_exp, MOR);

In [ ]:
valores_exp = collect(exp_filtrado[:, "School life expectancy, primary to tertiary, female (years)"]);

valores_mor = collect(mor_filtrado[:, "2010"]);

In [ ]:
palette = distinguishable_colors(93)
scatter(valores_exp, valores_mor, color=palette, legend=false, xlabel="Años esperados de estudio, mujeres", ylabel="Mortalidad infantil", title="Años de estudio y mortalidad infantil en varios países, 2010", titlefont=font(8))

Además de esta pequeña gráfica, hay evidencia que sugiere que la educación de las mujeres es un factor clave en la reducción de la mortalidad infantil.

Puede ser que los países con altos niveles de educación de las mujeres tengan tasas más bajas de mortalidad infantil en comparación con los países con bajos niveles de educación de las mujeres. Sin embargo, es importante tener en cuenta que otros factores también pueden influir en las tasas de mortalidad infantil, como la calidad y accesibilidad de la atención médica, la nutrición, la seguridad y la higiene, entre otros.

En cuanto al PIB per cápita, también puede tener un impacto en la mortalidad infantil. Los países con una economía más fuerte y una mayor inversión en salud pública y atención médica tienen más recursos para proporcionar atención de calidad a madres y niños. Sin embargo, también es importante tener en cuenta que la distribución del ingreso y la desigualdad económica pueden influir en la salud y la mortalidad infantil, incluso en países con altos niveles de PIB per cápita.

Así que en resumen, la educación de las mujeres y el PIB per cápita pueden influir en la mortalidad infantil, pero es importante reconocer que otros factores también pueden ser importantes y que la relación entre estos factores puede ser compleja y estar influenciada por una variedad de muchos más factores.

**3/5 pts (6 de 10)**

**Notas:**
- Muchos de los `for` que utilizan podrían estar dentro de una función y así su código sería más rápido, estaría mejor organizado y podrían evitar repetir código. 
- ¿Cuáles son los estados con mayor número de violaciones?
- En la gráfica `Años de estudio y mortalidad infantil en varios países, 2010`, les recomendaría unir los puntos con líneas para apreciar mejor la tendencia. Además, en la misma gráfica no se alcanzan a ver las etiquetas de los ejes, lo cual pueden solucionar jugando con las dimensiones de la imagen. 
- En su análisis del inciso (ii) primero dicen que a mayor PIB menor brecha, pero luego dicen que en economías más fuertes los puestos mejor pagados están dominados por hombres ¿Cuál es la interpretación correcta de su gráfica?
- En su gráfica `Porcentaje de mujeres en trabajos de paga baja y el producto interno bruto del país, 2018`, ¿observan alguna tendencia? Además, les recomendaría normalizar el eje $x$. 
- Está muy interesante su gráfica `Años de estudio y mortalidad infantil en varios países, 2010`. ¿Qué función le ajustarían?

## Problemas avanzados

Esta vez tengo menos problemas, así que los problemas avanzado comienzan en uno para sacar 8 (que con una mejora se puede volver un ejercicio para sacar 10) y termina en un ejercicio para sacar 11 de Maching Lerning. 



Para sacar 8: 

Obtén de <a href="https://smn.conagua.gob.mx/es/climatologia/informacion-climatologica/informacion-estadistica-climatologica">aquí</a> los datos de temperatura máxima y mínima diaria de alguna estación meteorológica con al menos 30 años de datos. Con esos datos has un archivo CSV (vía cualquier editor de texto, por ejemplo notepad). Lee la base de datos (que debería tener fecha, precipitación, evaporación, t_max y t_min). Utiliza LsqFit para ajustar una recta a los datos de temperatura máxima y a los datos de temperatura mínima como función del tiempo. Grafica los datos y las respectivas rectas ajustadas. Asegúrate que los datos que tomes en cuenta para el ajuste comiencen y terminen en la misma fecha del año. Por ejemplo del 15 de marzo de 1981 al 15 de marzo de 2021. Entre más datos, mejor, pero siempre cumpliendo con que terminen e inicien en el mismo día del año (sino puedes obtener pendientes negativas o positivas sólo porque tomaste de invierno a verano o vise versa). 
    
La ordenada al origen representa algo así como el valor inicial de la temperatura, mientras que la pendiente representa el cambio de temperatura en el tiempo. Si es positivo, entonces se está calentando en el tiempo el lugar donde se midió la temperatura. 
    
Ahora repite esto mismo para muchas estaciones, al menos 20 estaciones (aunque sería mejor unas 100). No es necesario que grafiques los datos de todas, pero te recomiendo hacerlo una vez para que elijas qué estación es más representativa de las conclusiones finales que tengas. Lo que sí necesitas hacer es obtener la ordenada al origen y la pendiente de la curva que mejor se ajuste. Recuerda que por cada estación obtienes 2 ordenadas al origen y dos pendientes, que representan lo de las temperaturas máximas y lo de las temperaturas mínimas. 
    
Finalmente, haz un histograma de tus pendientes ¿Notas que haya calentamiento? ¿cuantos grados por año se está calentando o enfriando en promedio las regiones que observaste?
    
**Nota:** El problema se vuelve de 10 si programas una forma automática de leer los datos de muchas estaciones (quizá así puedes revisar unas 1000 estaciones de golpe). Pista: Usa HTTP y nota que los archivos de la conagua están en páginas con el nombre del archivo y esos nombres siguen un código. El problema no es tan sencillo, porque tienes que eliminar varias filas del archivo que tienen información irrelevante para estos propósitos, además de que hay por ahí símbolos incómodos y el código debe también quedarse con las estaciones que tengan suficientes datos. 

Problema para sacar 9: 

Baja de [<em>aquí<em>](https://covid.ourworldindata.org/data/owid-covid-data.csv) la base de datos de covid para el mundo. 
    
(i) Haz una función que busque el primer día de contagio para cada país y obtenga los datos de los primeros 100 días desde el primer día de contagio (similar para las muertes). 

(ii) Ajusta usando LsqFit una exponencial ($A e^{\lambda x} + b$) y una ley de potencias ($m x^{\beta}+b$) a los datos de los primeros 100 días de pandemia en todos los países de America, Europa y Asia con más de 100 días de contagios. Oceanía lo descarto porque han tenido muy pocos casos en general, así que la estadística no es buena y África tuvieron pésimos registros durante casi toda la pandemia, así que los datos no son nada confiables. 

    
(iii) Haz dos gif animados, uno donde tengas un ajuste con una exponencial y otro donde tengas un ajuste con una ley de potencias para todos los países, graficando del número de casos totales como función del número de días desde el primer caso reportado y cuyo título sea el nombre del país. 
     
¿En general qué ajuste es mejor: el de crecimiento exponencial o el de ley de potencias?
    



In [ ]:
COV = CSV.File(open(read, "owid-covid-data.csv")) |> DataFrame;

In [ ]:
size(COV)

In [ ]:
function primer_caso(df::DataFrame, str::AbstractString)
    nfil, ncol = size(df)

    # iterar sobre la primera columna hasta que se encuentre el valor deseado
    for i = 1:nfil
        if df[i, 1] == str
            # a partir de esta fila, iterar en la siguiente columna
            for j = 5:5
                if !ismissing(df[i, j])
                    return df[i, j-1]
                end
            end
        end
    end

    # si no se encontró el valor deseado, o no se encontró un valor no faltante en la siguiente columna,
    # devolver missing
    return missing
end

In [ ]:
primer_caso(COV, "ARG"), primer_caso(COV, "MEX"), primer_caso(COV, "AFG")

In [ ]:
function primera_muerte(df::DataFrame, str::AbstractString)
    nfil, ncol = size(df)

    # iterar sobre la primera columna hasta que se encuentre el valor deseado
    for i = 1:nfil
        if df[i, 1] == str
            # a partir de esta fila, iterar en la siguiente columna
            for j = 8:8
                if !ismissing(df[i, j])
                    return df[i, j-4]
                end
            end
        end
    end

    # si no se encontró el valor deseado, o no se encontró un valor no faltante en la siguiente columna,
    # devolver missing
    return missing
end

In [ ]:
primera_muerte(COV, "ARG"), primera_muerte(COV, "MEX"), primera_muerte(COV, "AFG")

In [ ]:
nfil, ncol = size(COV)

In [ ]:

Paises = [] #Obtnemos todos los países de América, Asia y Europa que después de 100 días, tengan más de 100 casos
for i in 1:nfil
    for j in 4:4
        if COV[i, j] == Date(2020, 4, 13) && !ismissing(COV[i, j+1]) && 
            COV[i, j+1] >= 100 && !ismissing(COV[i, j-2]) && (COV[i, j-2] in ["South America", "North America", "Asia", "Europe"])
            push!(Paises, COV[i, j-1])
        end
    end
end

In [ ]:
Paises

In [ ]:
COV_100 = filter(row -> row."location" in Paises, COV) #El data Frame pero solo con los países que nos interesan.

In [ ]:
COV_NM = dropmissing(COV_100, :total_cases)

In [ ]:
# Creamos un vector con los valores de la columna mi_columna a partir de la fila 10
mi_lista = COV_NM[1:100, :total_cases]

# Convertimos el vector en una lista
mi_lista = collect(mi_lista)

In [ ]:
scatter(1:100, mi_lista)

In [ ]:
# Creamos un vector vacío para almacenar los resultados
resultados = Vector{DataFrame}(undef, length(Paises))

# Recorremos los valores de la lista
for i in 1:length(Paises)
    # Buscamos el valor en la primera columna del DataFrame
    idx = findfirst(COV_NM.location .== Paises[i])
    # Seleccionamos las 100 filas siguientes
    resultados[i] = COV_NM[idx:(idx+99), :]
end

# Unimos todos los resultados en un solo DataFrame
resultado_final = vcat(resultados...)

In [ ]:
todos_valores = collect(resultado_final.total_cases)

In [ ]:
# Creamos una lista de colores para cada iteración
colores = [:red, :green, :blue, :orange, :purple, :brown, :black]

# Creamos una figura vacía
scatter()

# Iteramos a través de la lista y graficamos los primeros 100 valores en cada iteración
for i in 1:107
    # Obtenemos los primeros 100 valores de la lista
    datos = todos_valores[(i-1)*100+1:i*100]
    
    # Graficamos los datos en el scatter
    scatter!(1:100, datos, color=colores[i%length(colores)+1])
end

# Mostramos la figura
plot!(legend=false)

##Obtenemos la gráifica de los primeros 100 días de casos a partir del primer caso registrado. Observamos que la curva más pronunciada 
#corresponde a Estados Unidos. Esto nos dice, que en los primeros días, a partir del primer caso en cada país, al que peor le fue, fue a 
#Estados Unidos.

**3 pts (9 de 10)**

**Notas:** Muy bien!
- En su función `primer_caso`, el `for` sobre `j`
```julia
for j = 5:5
    if !ismissing(df[i, j])
        return df[i, j-1]
    end
end
```
es innecesario. Lo mismo ocurre en `primera_muerte`. 
- En su última gráfica:
    - Les recomiendo cambiar la escala del eje $y$, por ejemplo, dividir todo entre $1\times10^6$.
    - ¿A qué país corresponde cada curva?, o al menos, las cuvas que más crecen. 
- Les faltó ajustar un modelo a alguna de las curvas. 

Problema para sacar 10: 

Haz un bot de Télegram (en un notebook aparte) que registre todas las palabras que utilice cada usuario (y por usuario), es decir, el bot debe generar una base de datos con las siguientes columnas: "Palabras", "ContadorNombreDeUsuario1", "ContadorNombreDeUsuario2", etc...(NombreDeUsuario$i$ debe de sustituirse por el nombre del usuario $i$) donde cada contador es el número de veces que ese usuario a usado la respectiva palabra. 

Añade tu bot (necesita ser administrador el bot, sino no puede responder los mensajes) a un grupo con amigos (o compañeros de equipo) y platica con tus amigos (compañeros) lo más "natural" posible (como si no existiera el bot) y tanto como puedas (por lo tanto, entre más cercanos sean tus amigos (compañeros) será mejor). Con cada frase que se ponga en el grupo el bot debe descomponerla en "palabras" (en español usando la paquetería StringEncodings) omitiendo signos de puntuación. De cada palabra, si una palabra no existe, la añade a la base de datos y pone el contador del usuario que la usó en 1 y los demás en 0. Si la palabra existe simplemente actualiza el contador para el usuario correspondiente. 

El bot debe además, con cada frase que se escriba, crear (o re-escribir) un archivo CSV que contenga la base de datos. 

También debes definir los comandos "\Graficar" y "\PalabrasComunes":

1. Si lo que se envía al bot es el comando "\Grafica", el bot debe regresar al grupo de télegram una gráfica de Zipf, es decir, en el eje $y$ la frecuencia de cada palabra dividido entre el conteo total de palabras de ese usuario, ordenado de mayor a menor la frecuencia de las palabras y en el eje $x$ los números 1,2,... que corresponden a la posición de cada palabra (la más frecuente, la segunda más frecuente, etc...). Ambos ejes deben estar en escala logarítmica, por lo que es necesario que si el contador de alguna palabra es 0, se sustituya por NaN. 
2. Si el comando que se envía al bot es "\PalabrasComunes" el bot debe de regresar "usuario1: palabra1usuario1, palabra2usuario1,...,palabra10usuario1", "usuario2:, palabra1usuario2,palabra2usuario2, ...alabra10usuario2" , ... con usuarioi el nombre del usuario i, y palabra$j$usuarioi la $j$-esima plabra más común del usuario $i$ (sólo las 10 primeras palabras).

Finalmente, en este notebook analiza la base de datos que hayas generado con el bot. Para cada usuario haz su gráfica de Zipf con un ajuste a una ley de potencias $a x^b$. Es importante antes de ajustar los datos ver la gráfica, pues típicamente la ley de potencias no se ajusta bien para todos los datos, así que hay que considerar sólo aquellos donde se ve "más o menos una recta" cuando se grafica en log-log (si tienes dudas, pregúntame vía télegram mostrándome la gráfica que obtuviste). 

Recuerda también que realmente se necesita una conversación larga. La palabra más común debería por lo menos aparecer $10^3 veces$ (es decir, en verdad les recomiendo elegir personas con las que puedan chismear a gusto para la parte de probar su bot). 

¿Qué valores de $b$ se obtiene para cada usuario? 


Problema alterno para sacar 11: 

Muy similar al problema anterior, pero en vez de buscar obtener la gráfica de Zipf, lo que buscarás es la probabilidad de una palabra dada otra. Es decir, se tiene que hacer una matriz donde el elemento $i,j$ es la probabilidad de que la palabra $i$ aparezca después de la palabra $j$. Conviene pre-hacer esta matriz con $0$'s inicialmente. Para esto uno tiene que tener un estimado del tamaño de la matriz. En español hay al rededor de $10^5$ palabras, pero coloquialmente usamos unas $2 \times 10^3$, así que podemos restringirnos a una matriz cuadrada de $(2 \times 10^3) \times (2 \times 10^3)$ elementos. Eso sigue siendo una matriz gigante y por lo tanto una base de datos gigante. Conviene hacerlo como una imagen y su respectivo diccionario. En vez de que la matriz nos diga la probabilidad, nos puede decir el número de veces que ha ocurrido la palabra $j$ después de la palabra $i$. 

El bot debe primero leer cada mensaje y descomponerlo en palabras y puntos (excluyendo todos los demás signos de puntuación, incluido "..."). Después debe separar el mensaje en fraces, es decir, obtener los arreglos de palabras que comienzan en la primera palabra y terminan en un punto  o la última palabra (lo que suceda primero), o que cominzan en la palabra después de un punto y terminan en otro punto o la última palabra (lo que suceda primero). Después debe buscar cada palabra en un arreglo de palabras $palabras$ que se tenga (inicialmente vacío). Si la palabra no se encuentra en $palabras$, entonces debe agregarse al arreglo. Después, cada frase se debe cambiar por un arreglo $indices$_$palabras$ de números naturales, de forma que si $j = indices$_$palabras[i]$, entonces $palabras[j]$ será la $i$-estima palabra de la frase. Finalmente, se debe buscar para cada elemento de $indices$_$palabras$ excepto el último, cuál es el valor $j = indices$_$palabras[i]$ y el valor $k = indices$_$palabras[i+1]$ y modificar la imagen de la base de datos, agregando un elemento al pixel $(j, k)$. 

Además el bot debe generar una lista de las palabras usadas, pero ordenadas de la más probable a la menos probable (para esto debe de haber una lista de la frecuencia de aparición de cada palabra).

Por último, usando esa base de datos se debe hacer una función que tenga como argumento una palabra y que arroje la palabra más probable que vendría después (si aún no hay una sugerencia, entonces debe proponer la palabra más frecuente). Si la palabra del argumento no está en la lista de $palabras$, entonces debe arrojar un error. 

Entrena tu función predictiva con una conversación o muchas conversaciones largas en telegram (tan largas como te sea posible). 

Luego, utiliza tu función para producir una frase de 20 palabras, comenzando con una palabra cualquiera de tu lista $palabras$ y después generando con eso una sugerencia de palabra que a su vez es el imput de tu función de su gerencia y así iterativamente 20 veces. ¿Sale una frase con sentido? 

**Nota:** Como pre-fijas el tamaño de la matriz, una vez que se el arreglo $palabras$ llega a $2\times 10^3$ se dejan de agregar elementos y las frases deben separarse no sólo por puntos, sino por cualquier palabra de que no se encuentre en $palabras$. 

**Nota2:** Si haces este problema, habrás hecho tu primer programa de Maching Lerning del curso. 